# **Install Packages**

In [1]:
%pip install PyMuPDF -q
%pip install gmft -q
%pip install sentence-transformers mistralai -q
%pip install mistralai --upgrade -q
%pip install langchain -q
%pip install langchain-community -q
%pip install langchain_experimental -q
%pip install -U langchain-huggingface -q
%pip install --upgrade langchain -q
%pip install tiktoken -q
%pip install ftfy -q
%pip install python-pptx python-docx -q
%pip install --upgrade pymilvus requests tqdm tf-keras -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# **Import Libraries**

In [1]:
import os
import json
import fitz  # PyMuPDF
import ftfy
import tiktoken
import collections
from docx import Document
from pptx import Presentation
from gmft.auto import AutoTableDetector, AutoTableFormatter
from gmft.pdf_bindings import PyPDFium2Document

# Configration 
detector = AutoTableDetector()
formatter = AutoTableFormatter()

tesseract_path = os.path.join(os.getcwd(), ".venv", "Tesseract-OCR", "tesseract.exe")
os.environ["TESSERACT_PATH"] = tesseract_path

DPI = 150  # For OCR resolution

e:\rag_app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Document Parsing**

In [2]:
def is_page_searchable(page, min_chars=10):
    return len(page.get_text().strip()) >= min_chars

def process_ocr(page, dpi=150, language=None):
    pix = page.get_pixmap(dpi=dpi)
    ocr_pdf_bytes = pix.pdfocr_tobytes(language=language)
    return fitz.open("pdf", ocr_pdf_bytes)

def extract_tables(pdf_source, page_number):
    doc = PyPDFium2Document(pdf_source)
    tables = []
    try:
        page = doc[page_number]
        for cropped in detector.extract(page):
            formatted = formatter.extract(cropped, margin="auto", padding=None)
            df = formatted.df()
            tables.append((cropped.bbox, df))
    finally:
        doc.close()
    return tables

def extract_tables_from_doc(fitz_doc, page_number):
    single_page_doc = fitz.open()
    single_page_doc.insert_pdf(fitz_doc, from_page=page_number, to_page=page_number)
    pdf_bytes = single_page_doc.tobytes()
    single_page_doc.close()
    return extract_tables(pdf_bytes, 0)

def uniquify_columns(cols):
    counts = collections.Counter()
    new_cols = []
    for col in cols:
        counts[col] += 1
        new_cols.append(col if counts[col] == 1 else f"{col}_{counts[col] - 1}")
    return new_cols

# **Chunking and Text Processing**

In [3]:
def process_text_utf8(text):
    if not isinstance(text, str):
        text = str(text)
    text = ftfy.fix_text(text)
    text = text.replace("\x00", "").replace("\ufeff", "")
    text = text.encode("utf-8", errors="ignore").decode("utf-8")
    text = text.replace('\n', '').replace('\"', '')
    return text.strip()


def count_tokens(text):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    return len(tokenizer.encode(text))

def custom_chunking(text, chunk_size=500, overlap=50):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    token_ids = tokenizer.encode(text)
    chunks, start = [], 0
    while start < len(token_ids):
        end = min(start + chunk_size, len(token_ids))
        chunk_token_ids = token_ids[start:end]
        chunks.append(tokenizer.decode(chunk_token_ids))
        start += chunk_size - overlap
    return chunks

def chunking_workflow(text, max_tokens=500, overlap=50):
    clean_text = process_text_utf8(text)
    if not clean_text.strip():
        return []
    token_count = count_tokens(clean_text)
    if token_count <= max_tokens:
        return [{"text": clean_text, "metadata": {}, "token_count": token_count}]
    chunks = custom_chunking(clean_text, chunk_size=max_tokens, overlap=overlap)
    return [
        {"text": chunk, "metadata": {}, "token_count": count_tokens(chunk)}
        for chunk in chunks
    ]

def create_elements_with_metadata(chunks, tables, input_file, text_positions=None, table_positions=None):
    elements = []
    base_name = os.path.basename(input_file)
    # Add text chunks with position if available
    for i, chunk in enumerate(chunks, 1):
        metadata = chunk.get("metadata", {})
        metadata.update({
            "source_document": base_name, "chunk_id": i, 
            "position": text_positions[i-1] if text_positions and i-1 < len(text_positions) else None})
        elements.append({"type": "text","content": chunk['text'],"metadata": metadata,"token_count": chunk['token_count']})

    # Add tables with position if available
    for j, table in enumerate(tables, start=i+1):
        if isinstance(table, dict) and "metadata" in table:
            # PDF table with metadata
            meta = table["metadata"].copy()
            meta.update({"source_document": base_name,"chunk_id": j})
            content = table.get("content", table)
        else:
            meta = {
                "source_document": base_name,"chunk_id": j, 
                "position": table_positions[j - (i+1)] if table_positions and (j - (i+1)) < len(table_positions) else j}
            content = table
        elements.append({"type": "table","content": content,"metadata": meta})

    return elements

In [4]:
def process_pdf_pages(input_pdf):
    src = fitz.open(input_pdf)
    ocr_doc = fitz.open()
    tables, text_blocks = [], []
    text_positions, table_positions = [], []
    filename = os.path.basename(input_pdf)

    for i, page in enumerate(src):
        print(f"   📄 Processing Page {i + 1}/{len(src)}")
        searchable = is_page_searchable(page)
        temp_doc = None  # To manage OCR temp doc lifetime
        if not searchable:
            print(f"      🔎 Searchable: ❌ No, requires OCR")
            temp_doc = process_ocr(page, dpi=DPI)
            blocks_page = temp_doc[0]
            page_tables = extract_tables_from_doc(temp_doc, 0)
            ocr_doc.insert_pdf(temp_doc)
        else:
            print(f"      🔎 Searchable: ✅ Yes")
            blocks_page = page
            page_tables = extract_tables_from_doc(src, i)

        for idx, (bbox, df) in enumerate(page_tables, 1):
            df.columns = uniquify_columns(df.columns.astype(str))
            tables.append({
                "type": "table",
                "content": df.to_dict(orient="records"),
                "metadata": {"page_number": i + 1,"columns": df.columns.tolist(),
                    "table_index_on_page": idx,"position": bbox[1], "source_document": filename}})
            table_positions.append(bbox[1])  # Save y-position

        table_boxes = [box for box, _ in page_tables]
        for block in blocks_page.get_text("blocks"):
            if len(block) >= 5:
                x0, y0, x1, y1, text = block[:5]
                rect = fitz.Rect(x0, y0, x1, y1)
                if not any(fitz.Rect(*box).intersects(rect) for box in table_boxes) and text.strip():
                    text_blocks.append((i, y0, text.strip()))
                    text_positions.append(y0)  # Save y-position

    text_blocks.sort(key=lambda x: (x[0], x[1]))
    full_text = "\n".join(text for _, _, text in text_blocks)
    chunks = chunking_workflow(full_text)
    chunk_positions = text_positions[:len(chunks)]
    elements = create_elements_with_metadata(
        chunks, [t for t in tables], input_pdf, text_positions=chunk_positions, table_positions=table_positions
    )
    return src, ocr_doc, elements


def extract_text_and_tables_from_docx(path):
    doc = Document(path)
    text = [para.text.strip() for para in doc.paragraphs if para.text.strip()]
    tables = []
    for table in doc.tables:
        table_data = [[cell.text.strip() for cell in row.cells] for row in table.rows]
        tables.append(table_data)
    return text, tables

def extract_text_and_tables_from_pptx(path):
    prs = Presentation(path)
    text, tables = [], []
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                text.append(shape.text.strip())
            if hasattr(shape, "has_table") and shape.has_table:
                table = shape.table
                table_data = [[cell.text_frame.text.strip() if cell.text_frame else "" for cell in row.cells] for row in table.rows]
                tables.append(table_data)
    return text, tables


def save_processed_output(src, ocr_doc, elements, ocr_pdf_path):
    texts = [e for e in elements if e['type'] == 'text']
    tables = [e for e in elements if e['type'] == 'table']

    if texts:
        with open("text_output.json", "w", encoding="utf-8") as f:
            json.dump({"text_chunks": texts}, f, indent=2, ensure_ascii=False)
    if tables:
        with open("tables_output.json", "w", encoding="utf-8") as f:
            json.dump({"tables": tables}, f, indent=2, ensure_ascii=False)
    if ocr_doc and len(ocr_doc) > 0 and ocr_pdf_path:
        ocr_doc.save(ocr_pdf_path)

    if ocr_doc: ocr_doc.close()
    if src: src.close()

    return {"text_chunks": texts, "tables": tables}


def smart_file_processing(input_file, ocr_output_pdf="ocr_output.pdf"):
    if not os.path.exists(input_file):
        print(f"❌ File not found: {input_file}")
        return

    ext = os.path.splitext(input_file)[1].lower()
    print(f"📁 Processing file: {input_file}")

    if ext == ".pdf":
        src, ocr_doc, elements = process_pdf_pages(input_file)
        save_processed_output(src, ocr_doc, elements, ocr_output_pdf)

    elif ext == ".docx":
        texts, tables = extract_text_and_tables_from_docx(input_file)
        full_text = "\n".join(texts)
        chunks = chunking_workflow(full_text)
        elements = create_elements_with_metadata(
        chunks, tables, input_file, text_positions=list(range(len(chunks))), 
        table_positions=list(range(len(tables))))
        save_processed_output(None, None, elements, None)

    elif ext == ".pptx":
        texts, tables = extract_text_and_tables_from_pptx(input_file)
        full_text = "\n".join(texts)
        chunks = chunking_workflow(full_text)
        elements = create_elements_with_metadata(
        chunks, tables, input_file,text_positions=list(range(len(chunks))),
        table_positions=list(range(len(tables))))
        save_processed_output(None, None, elements, None)

    else:
        print(f"❌ Unsupported file type: {ext}")

    print("\n✅ Done!")


In [6]:
input_file = r"..\data\Shell - Financial Statement-Page1-5 1.pdf" 
smart_file_processing(input_file)

📁 Processing file: ..\data\Shell - Financial Statement-Page1-5 1.pdf
   📄 Processing Page 1/5
      🔎 Searchable: ❌ No, requires OCR
   📄 Processing Page 2/5
      🔎 Searchable: ❌ No, requires OCR
   📄 Processing Page 3/5
      🔎 Searchable: ❌ No, requires OCR
   📄 Processing Page 4/5
      🔎 Searchable: ❌ No, requires OCR
   📄 Processing Page 5/5
      🔎 Searchable: ❌ No, requires OCR

✅ Done!


In [7]:
import json

def print_tokens_per_chunk(file_path="text_output.json"):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    chunks = data.get("text_chunks", [])
    for idx, chunk in enumerate(chunks):
        chunk_type = chunk.get("type", "unknown")
        token_count = chunk.get("token_count", 0)
        print(f"Chunk {idx+1} | Type: {chunk_type} | Tokens: {token_count}")

    print(f"\nTotal Chunks: {len(chunks)}")

print_tokens_per_chunk("text_output.json")

Chunk 1 | Type: text | Tokens: 311

Total Chunks: 1


# **Milvus Vector DB**

In [20]:
import json
from tqdm import tqdm
from dotenv import load_dotenv
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, list_collections
from mistralai import Mistral
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')
# --- 1. SETUP ---

# ⚙️ Connect to Milvus
print("🔌 Connecting to Milvus...")
connections.connect("default", host="localhost", port="19530")
print("✅ Connected to Milvus.")

# 🤖 Load Embedding Model
print("🤖 Loading sentence transformer model...")
text_encoder = SentenceTransformer("all-MiniLM-L6-v2")
print("✅ Model loaded.")

# --- 2. SCHEMA & COLLECTION DEFINITION ---

# 🏗 Define a robust schema that includes a field for the original content
common_fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="page_no", dtype=DataType.INT64),
    FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=65535),
]
text_schema = CollectionSchema(fields=common_fields, description="Text document chunks")
table_schema = CollectionSchema(fields=common_fields, description="Table document chunks")

# 🗃 Create/Load Collections
print("🗃 Creating/Loading Milvus collections...")

TEXT_COLLECTION_NAME = "textcollections"
TABLE_COLLECTION_NAME = "tablecollections"


if TEXT_COLLECTION_NAME not in list_collections():
    text_col = Collection(TEXT_COLLECTION_NAME, schema=text_schema)
    print("✅ Text Collection created.")
else:
    text_col = Collection(name=TEXT_COLLECTION_NAME) # Correctly assign here
    print("📁 Text Collection already exists.")

if TABLE_COLLECTION_NAME not in list_collections():
    table_col = Collection(TABLE_COLLECTION_NAME, schema=table_schema)
    print("✅ Table Collection created.")
else:
    table_col = Collection(name=TABLE_COLLECTION_NAME) # And here
    print("📁 Table Collection already exists.")

print(f"✅ Collections '{text_col.name}' and '{table_col.name}' are ready.")

# --- 3. CORE FUNCTIONS ---

def embed_and_insert(collection, data, encoder, data_type):
    """
    Embeds content and inserts it into a Milvus collection.
    Handles different JSON structures and serializes table content.
    """
    if isinstance(data, dict):
        data = [data]
    if not isinstance(data, list):
        print(f"⚠️ Data for '{data_type}' is not a processable list. Skipping insertion.")
        return

    embeddings, sources, pages, types, contents = [], [], [], [], []
    print(f"📦 Preparing '{data_type}' data for insertion...")
    for i, chunk in enumerate(tqdm(data, desc=f"Embedding {data_type}s")):
        content_str = ""
        metadata = {}
        if isinstance(chunk, dict):
            possible_keys = ["content", "text", "page_content"]
            for key in possible_keys:
                if chunk.get(key):
                    content_str = chunk[key]
                    break
            metadata = chunk.get("metadata", {})
        elif isinstance(chunk, str):
            content_str = chunk
        
        if isinstance(content_str, list): # Serialize table data
            content_str = json.dumps(content_str)
        if not content_str.strip():
            continue
            
        source = metadata.get("source_document", "unknown")
        try:
            page_no = int(metadata.get("chunk_id", i))
        except (ValueError, TypeError):
            page_no = i

        emb = encoder.encode(content_str)
        
        embeddings.append(emb.tolist())
        sources.append(source)
        pages.append(page_no)
        types.append(data_type)
        contents.append(content_str)

    if not embeddings:
        print(f"⚠️ No valid '{data_type}' data found to insert.")
        return

    print(f"🚀 Inserting {len(embeddings)} '{data_type}' vectors into '{collection.name}'...")
    collection.insert([embeddings, sources, pages, types, contents])
    collection.flush()
    print(f"✅ Data successfully inserted and flushed into '{collection.name}'.")

# --- 4. Retrieve results ---

def retrieve(query, top_k=5):
    """Searches both text and table collections, then combines and re-ranks the results."""

    query_vec = text_encoder.encode(query).tolist()
    text_col.load()
    table_col.load()
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    
    text_results = text_col.search(
        data=[query_vec], 
        anns_field="embedding", 
        param=search_params, 
        limit=top_k,
        output_fields=["source", "page_no", "type", "content"]
    )
    table_results = table_col.search(
        data=[query_vec], 
        anns_field="embedding", 
        param=search_params, 
        limit=top_k,
        output_fields=["source", "page_no", "type", "content"]
    )
    
    combined = {}
    all_results = text_results[0] + table_results[0]
    for res in all_results:
        key = (res.entity.get("source"), res.entity.get("page_no"))
        if key not in combined or combined[key].distance > res.distance:
            combined[key] = res
            
    return sorted(list(combined.values()), key=lambda x: x.distance)

# ---- 5. Generate answer ---

def rag_answer(query):
    """Performs the full RAG pipeline: retrieve, prompt, and generate.
    Returns a tuple of (answer, retrieved_chunks) where retrieved_chunks is a list of dicts
    containing content, source, page_no, and similarity score."""
    
    retrieved_hits = retrieve(query, top_k=5)

    if not retrieved_hits:
        return "I could not find any relevant information in the documents to answer your question.", []

    # Prepare the context block for generation
    context_block = "\n\n---\n\n".join([hit.entity.get("content", "") for hit in retrieved_hits])
    
    prompt = (
        "You are an expert AI assistant. Use only the provided context below to answer the user's question. "
        "Your answer must be based solely on this context. If the context does not contain the answer, "
        "state that you cannot answer based on the provided information.\n\n"
        f"--- CONTEXT ---\n{context_block}\n\n--- END CONTEXT ---\n\n"
        f"Question: {query}\nAnswer:"
    )
    
    load_dotenv()
    api_key = os.getenv("MISTRAL_API_KEY")
    if not api_key:
        raise ValueError("MISTRAL_API_KEY not found. Please create a .env file with your key.")
        
    client = Mistral(api_key=api_key)

    print("🤖 Generating answer with Mistral AI...")
    response = client.chat.complete(
        model="mistral-small-latest",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
    )
    print("✅ Answer generated.")
    
    # Prepare the retrieved chunks information
    retrieved_chunks = []
    for hit in retrieved_hits:
        chunk_info = {
            "content": hit.entity.get("content", ""),
            "source": hit.entity.get("source", "unknown"),
            "page_no": hit.entity.get("page_no", 0),
            "similarity_score": 1 - hit.distance,  # Convert distance to similarity (higher is better)
            "type": hit.entity.get("type", "unknown")
        }
        retrieved_chunks.append(chunk_info)
    
    return response.choices[0].message.content.strip(), retrieved_chunks

🔌 Connecting to Milvus...
✅ Connected to Milvus.
🤖 Loading sentence transformer model...
✅ Model loaded.
🗃 Creating/Loading Milvus collections...
📁 Text Collection already exists.
📁 Table Collection already exists.
✅ Collections 'textcollections' and 'tablecollections' are ready.


In [21]:
# --- Load Data ---
print("\n📥 Loading JSON data...")
try:
    with open("text_output.json", "r", encoding="utf-8") as f:
        text_data = json.load(f)["text_chunks"]
    with open("tables_output.json", "r", encoding="utf-8") as f:
        table_data = json.load(f)["tables"]
    print("✅ JSON data loaded successfully.")
except (FileNotFoundError, KeyError) as e:
    print(f"❌ Error loading data: {e}. Please ensure your JSON files exist and are correctly formatted.")
    exit()

# --- Ingest Data ---
embed_and_insert(collection=text_col, data=text_data, encoder=text_encoder, data_type="text")
embed_and_insert(collection=table_col, data=table_data, encoder=text_encoder, data_type="table")

# --- Create Indexes ---
print("\n🏗 Creating indexes for collections (if they don't exist)...")
index_params = {"metric_type": "L2", "index_type": "IVF_FLAT", "params": {"nlist": 128}}
if not text_col.has_index():
    text_col.create_index(field_name="embedding", index_params=index_params)
    print(f"✅ Index created for '{text_col.name}'.")
else:
    print(f"✅ Index already exists for '{text_col.name}'.")

if not table_col.has_index():
    table_col.create_index(field_name="embedding", index_params=index_params)
    print(f"✅ Index created for '{table_col.name}'.")
else:
    print(f"✅ Index already exists for '{table_col.name}'.")
    
# --- Verify & Query ---
text_col.flush()
table_col.flush()
print("\n" + "="*50)
print("📊 Collection Status:")
print(f"  - Entities in '{text_col.name}': {text_col.num_entities}")
print(f"  - Entities in '{table_col.name}': {table_col.num_entities}")
print("="*50)


📥 Loading JSON data...
✅ JSON data loaded successfully.
📦 Preparing 'text' data for insertion...


Embedding texts: 100%|██████████| 30/30 [00:02<00:00, 13.59it/s]


🚀 Inserting 30 'text' vectors into 'textcollections'...
✅ Data successfully inserted and flushed into 'textcollections'.
📦 Preparing 'table' data for insertion...


Embedding tables: 100%|██████████| 9/9 [00:00<00:00, 14.18it/s]


🚀 Inserting 9 'table' vectors into 'tablecollections'...
✅ Data successfully inserted and flushed into 'tablecollections'.

🏗 Creating indexes for collections (if they don't exist)...
✅ Index already exists for 'textcollections'.
✅ Index already exists for 'tablecollections'.

📊 Collection Status:
  - Entities in 'textcollections': 93
  - Entities in 'tablecollections': 33


In [23]:
query = "What are the two main components of the Pixtral architecture?"

answer, chunks = rag_answer(query)

print("\n" + "-"*50)
print(f"❓ Query: {query}")
print(f"💡 Answer: {answer}")
print("\n🔍 Retrieved Chunks:")
for i, chunk in enumerate(chunks, 1):
    print(f"\nChunk {i}:")
    print(f"Source: {chunk['source']} (Page {chunk['page_no']})")
    print(f"Type: {chunk['type']}")
    print(f"Similarity (out of 10): {chunk['similarity_score'] * 10:.2f}")
    print(f"Content: {chunk['content'][:200]}...") 
print("-"*50)

🤖 Generating answer with Mistral AI...
✅ Answer generated.

--------------------------------------------------
❓ Query: What are the two main components of the Pixtral architecture?
💡 Answer: The two main components of the Pixtral architecture are:
1. A **vision encoder**, which tokenizes images.
2. A **multimodal decoder**, which predicts the next text token given a sequence of text and images.

🔍 Retrieved Chunks:

Chunk 1:
Source: mistral.pdf (Page 5)
Type: text
Similarity (out of 10): 0.04
Content: relative position encodings lend themselves naturally to variable image sizes.
3
Figure 3: Complete Pixtral Architecture. Pixtral has two components: a vision encoder, which tokenizes
images, and a mu...

Chunk 2:
Source: mistral.docx (Page 12)
Type: text
Similarity (out of 10): -0.23
Content:  performance across various benchmarks, outperforming other open models and matching larger models. Its superior instruction following abilities, support for variable image sizes, and long context 